In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint
import json

In [9]:
path1 = ("Resources/filtered_covid.csv")
path2 = ("Resources/real_estate19.csv")
path3 = ("Resources/real_estate20.csv")

covid= pd.read_csv(path1)
rs_19= pd.read_csv(path2)
rs_20= pd.read_csv(path3)

In [10]:
covid = covid[(covid['YEAR']==2020) & (covid['MONTH']>5)]
covid

,GENDER,AGEGRP,DATE,COVID_COUNT,COUNTY_NAME,YEAR,MONTH,DAY,MONTH_NAME
5,F,0-19,2020-06-11 00:00:00,1,Boone,2020,6,11,June
6,F,0-19,2020-06-19 00:00:00,1,Boone,2020,6,19,June
7,F,0-19,2020-06-20 00:00:00,1,Boone,2020,6,20,June
8,F,0-19,2020-06-22 00:00:00,1,Boone,2020,6,22,June
9,F,0-19,2020-06-25 00:00:00,1,Boone,2020,6,25,June
...,...,...,...,...,...,...,...,...,...
19438,Unknown,Unknown,2020-08-25 00:00:00,1,Marion,2020,8,25,August
19439,Unknown,Unknown,2020-08-26 00:00:00,1,Marion,2020,8,26,August
19440,Unknown,Unknown,2020-10-24 00:00:00,1,Marion,2020,10,24,October
19441,Unknown,Unknown,2020-11-13 00:00:00,1,Marion,2020,11,13,November


In [11]:
covid_county_count= covid.groupby("COUNTY_NAME")["COVID_COUNT"].sum()
covid_county_count

COUNTY_NAME
Boone         4228
Hamilton     23086
Hancock       4886
Hendricks    10502
Johnson      11150
Marion       61713
Name: COVID_COUNT, dtype: int64

In [12]:
covid_month_county= covid.groupby("MONTH_NAME")["COVID_COUNT"].sum()
covid_month_county

MONTH_NAME
August        6877
December     48027
July          5314
June          2452
November     36572
October      10884
September     5439
Name: COVID_COUNT, dtype: int64

In [16]:
covid_month_stats= covid.groupby(["COUNTY_NAME", "MONTH"])["COVID_COUNT"].sum()
covid_month_stats

COUNTY_NAME  MONTH
Boone        6          108
             7          206
             8          219
             9          163
             10         417
             11        1338
             12        1777
Hamilton     6          331
             7          956
             8         1505
             9         1063
             10        2082
             11        7542
             12        9607
Hancock      6           81
             7          187
             8          206
             9          140
             10         416
             11        1582
             12        2274
Hendricks    6          192
             7          367
             8          613
             9          454
             10         888
             11        3275
             12        4713
Johnson      6          116
             7          396
             8          406
             9          370
             10        1183
             11        3682
             12        4997
M

In [6]:
covid_county_df= pd.DataFrame(covid_county_count)
covid_county_df

,COVID_COUNT
COUNTY_NAME,
Boone,4228
Hamilton,23086
Hancock,4886
Hendricks,10502
Johnson,11150
Marion,61713


In [17]:
covid_month_df= pd.DataFrame(covid_month_stats)
covid_month_df

COVID_COUNT
COUNTY_NAME MONTH             
Boone       6              108
            7              206
            8              219
            9              163
            10             417
            11            1338
            12            1777
Hamilton    6              331
            7              956
            8             1505
            9             1063
            10            2082
            11            7542
            12            9607
Hancock     6               81
            7              187
            8              206
            9              140
            10             416
            11            1582
            12            2274
Hendricks   6              192
            7              367
            8              613
            9              454
            10             888
            11            3275
            12            4713
Johnson     6              116
            7              396
            8              406
            9              370
            10            1183
            11            3682
            12            4997
Marion      6             1624
            7             3202
            8             3928
            9             3249
            10            5898
            11           19153
            12           24659